In [14]:
#Setup + Data Loading

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import io


# 1.2 Set seed and device
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 1.3 Dataset path
lfw_path = "../Datasets/lfw-dataset"

# 1.4 Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # For ResNet input
    transforms.ToTensor(),
])

# 1.5 Load dataset
lfw_dataset = datasets.ImageFolder(root=lfw_path, transform=transform)
lfw_loader = DataLoader(lfw_dataset, batch_size=16, shuffle=True)


Using device: cpu


In [15]:
#Define + Train ResNet (small batch for demo)

model = models.resnet18(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, len(lfw_dataset.classes))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.train()
for batch_idx, (images, labels) in enumerate(lfw_loader):
    if batch_idx >= 30: break
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    print(f"Batch {batch_idx+1}/30 - Loss: {loss.item():.4f}")

Batch 1/30 - Loss: 9.0540
Batch 2/30 - Loss: 8.7226
Batch 3/30 - Loss: 8.8491
Batch 4/30 - Loss: 9.0044
Batch 5/30 - Loss: 8.3634
Batch 6/30 - Loss: 9.2350
Batch 7/30 - Loss: 9.0577
Batch 8/30 - Loss: 8.8550
Batch 9/30 - Loss: 8.5327
Batch 10/30 - Loss: 8.9819
Batch 11/30 - Loss: 9.5624
Batch 12/30 - Loss: 10.0737
Batch 13/30 - Loss: 9.0845
Batch 14/30 - Loss: 8.7513
Batch 15/30 - Loss: 9.3882
Batch 16/30 - Loss: 8.7689
Batch 17/30 - Loss: 9.1567
Batch 18/30 - Loss: 8.9601
Batch 19/30 - Loss: 9.3708
Batch 20/30 - Loss: 9.5732
Batch 21/30 - Loss: 9.2405
Batch 22/30 - Loss: 8.7530
Batch 23/30 - Loss: 8.5632
Batch 24/30 - Loss: 9.3274
Batch 25/30 - Loss: 9.3998
Batch 26/30 - Loss: 9.2444
Batch 27/30 - Loss: 10.0054
Batch 28/30 - Loss: 9.1901
Batch 29/30 - Loss: 8.7757
Batch 30/30 - Loss: 9.6213


In [16]:
#FGSM Attack (loop until model gets one right)

def fgsm_attack(image, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed = image + epsilon * sign_data_grad
    return torch.clamp(perturbed, 0, 1)

model.eval()
image_found = False

for _ in range(30):
    image, label = next(iter(lfw_loader))
    image = image[0].unsqueeze(0).to(device)
    label = torch.tensor([label[0]]).to(device)
    image.requires_grad = True

    output = model(image)
    init_pred = output.argmax(dim=1)

    if init_pred.item() == label.item():
        print("✅ Found correct prediction:", lfw_dataset.classes[init_pred.item()])
        loss = criterion(output, label)
        model.zero_grad()
        loss.backward()
        data_grad = image.grad.data
        epsilon = 0.1
        perturbed_image = fgsm_attack(image, epsilon, data_grad)
        clean_pred = init_pred.item()
        break

if not image_found:
    print("❌ Could not find a correctly classified image.")
    

✅ Found correct prediction: George_W_Bush
❌ Could not find a correctly classified image.


In [8]:
#JPEG Defence
def jpeg_defence(image_tensor, quality=30):
    pil_img = transforms.ToPILImage()(image_tensor.squeeze().cpu())
    buffer = io.BytesIO()
    pil_img.save(buffer, format='JPEG', quality=quality)
    buffer.seek(0)
    compressed_img = Image.open(buffer)
    return transforms.ToTensor()(compressed_img).unsqueeze(0).to(device)

jpeg_image = jpeg_defence(perturbed_image)

model.eval()
with torch.no_grad():
    adv_out = model(perturbed_image)
    jpeg_out = model(jpeg_image)

adv_pred = adv_out.argmax(dim=1).item()
jpeg_pred = jpeg_out.argmax(dim=1).item()

print(f"🎯 Clean Prediction: {lfw_dataset.classes[clean_pred]}")
print(f"⚠️  Adversarial Prediction: {lfw_dataset.classes[adv_pred]}")
print(f"🔧 JPEG Defence Prediction: {lfw_dataset.classes[jpeg_pred]}")


NameError: name 'perturbed_image' is not defined